In [ ]:
%%capture
!pip install datasets --no-build-isolation
!pip install seqeval
!pip install transformers[torch]

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import torch
import numpy as np
from datasets import load_metric

# CoNLL-2003 Dataset Description

The CoNLL-2003 dataset is widely used for training and evaluating Named Entity Recognition (NER) models. The dataset focuses on four types of named entities: persons (PER), locations (LOC), organizations (ORG), and miscellaneous entities (MISC).

## Dataset Structure:
Each data file contains four columns separated by a single space:
1. Word
2. Part-of-Speech (POS) tag
3. Syntactic chunk tag
4. Named entity tag

Words are listed on separate lines, and sentences are separated by a blank line.
The chunk and named entity tags follow the IOB2 tagging scheme:
- `B-TYPE`: Beginning of a phrase of type TYPE
- `I-TYPE`: Inside a phrase of type TYPE
- `O`: Outside any named entity phrase

## Example:
```python
{
    "chunk_tags": [11, 12, 12, 21, 13, 11, 11, 21, 13, 11, 12, 13, 11, 21, 22, 11, 12, 17, 11, 21, 17, 11, 12, 12, 21, 22, 22, 13, 11, 0],
    "id": "0",
    "ner_tags": [0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "pos_tags": [12, 22, 22, 38, 15, 22, 28, 38, 15, 16, 21, 35, 24, 35, 37, 16, 21, 15, 24, 41, 15, 16, 21, 21, 20, 37, 40, 35, 21, 7],
    "tokens": ["The", "European", "Commission", "said", "on", "Thursday", "it", "disagreed", "with", "German", "advice", "to", "consumers", "to", "shun", "British", "lamb", "until", "scientists", "determine", "whether", "mad", "cow", "disease", "can", "be", "transmitted", "to", "sheep", "."]
}


## Named Entity Tags
- **O**: Outside a named entity
- **B-PER**: Beginning of a person's name
- **I-PER**: Inside a person's name
- **B-ORG**: Beginning of an organization name
- **I-ORG**: Inside an organization name
- **B-LOC**: Beginning of a location name
- **I-LOC**: Inside a location name
- **B-MISC**: Beginning of miscellaneous entity
- **I-MISC**: Inside a miscellaneous entity


In [ ]:
# Load dataset
dataset = load_dataset("conll2003")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
# Model checkpoint
checkpoint = "bert-base-cased"
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

## Token Labeling in NER: Use of `-100`

In Named Entity Recognition (NER) tasks, the label `-100` is commonly used to signify that certain tokens should be ignored during the loss calculation in model training. This approach helps focus the learning on meaningful parts of the data. Here's an overview of the types of tokens typically assigned a `-100` label:

### 1. **Subsequent Sub-tokens**
After a word is split into multiple sub-tokens, only the first sub-token receives the actual entity label. Subsequent sub-tokens receive `-100` to ensure that entity labels are not incorrectly assigned to fragments of words.

### 2. **Special Tokens**
Special tokens such as `[CLS]`, `[SEP]`, and `[PAD]` used for managing sequence boundaries and lengths in models like BERT are also assigned `-100` as they do not correspond to real words in the text.

### 3. **Non-Entity Tokens**
In certain training setups, tokens that do not correspond to any entity and are not the focus of the task might also be marked with `-100`, especially in cases of imbalanced datasets.

### Example
- **Sentence**: "John lives in New York"
- **Tokens**: ["[CLS]", "John", "lives", "in", "New", "York", "[SEP]"]
- **Labels**: [-100, "B-PER", "O", "O", "B-LOC", "I-LOC", -100]

This labeling strategy is critical for efficient model training, ensuring that the model focuses only on relevant tokens.


In [ ]:
# Tokenize and align labels without fixed padding
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
# Load pre-trained model
model = AutoModelForTokenClassification.from_pretrained(checkpoint, num_labels=9)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


`seqeval` is a Python framework for sequence labeling evaluation. seqeval can evaluate the performance of chunking tasks such as named-entity recognition, part-of-speech tagging, semantic role labeling and so on.

In [ ]:
# Load seqeval metric
metric = load_metric("seqeval")

# Define compute_metrics function
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
# Get label list
label_list = dataset["train"].features["ner_tags"].feature.names

# Set up data collator for dynamic padding
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
# Initialize Trainer with compute_metrics and data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# Train model
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.164800,0.041211,0.927306,0.933861,0.930572,0.988669
2,0.027900,0.039234,0.940369,0.944800,0.942579,0.990363
3,0.014300,0.037464,0.941855,0.948670,0.945250,0.990908


TrainOutput(global_step=2634, training_loss=0.05219505968289386, metrics={'train_runtime': 555.0062, 'train_samples_per_second': 75.896, 'train_steps_per_second': 4.746, 'total_flos': 1050534559887048.0, 'train_loss': 0.05219505968289386, 'epoch': 3.0})

In [ ]:
# Evaluate model
results = trainer.evaluate()
print(results)

{'eval_loss': 0.03746436536312103, 'eval_precision': 0.9418546365914787, 'eval_recall': 0.9486704813194211, 'eval_f1': 0.9452502724909868, 'eval_accuracy': 0.9909076749347767, 'eval_runtime': 10.5735, 'eval_samples_per_second': 307.372, 'eval_steps_per_second': 19.294, 'epoch': 3.0}
